In [53]:
# imports
import os
import pickle
import torch
import importlib
import constants
import utils
from simulator.nets import FeedForward
importlib.reload(constants)
importlib.reload(utils)

<module 'utils' from 'A:\\Dropbox\\eBay\\repo\\utils.py'>

### Attempting to load model

In [6]:
path = 'data/models'
os.chdir('A:/dropbox/ebay/') # BARRY'S Working directory

In [70]:
# loads a model given its name
def load_model(model_name):
    file = '{}/{}_1.pkl'.format(path, model_name)
    output = None
    output = pickle.load(open(file, 'rb'))
    return output
# gets types of everything in dict
def dict_types(input_dict):
    for key, value in input_dict.items():
        print('{}: {}'.format(key, type(value)))
    

In [8]:
# arrival bin
arrival_bin = load_model('arrival_bin')

In [12]:
print(type(arrival_bin))
print(len(arrival_bin))
for el in arrival_bin:
    print(type(el))

<class 'list'>
3
<class 'dict'>
<class 'numpy.ndarray'>
<class 'datetime.timedelta'>


In [16]:
ab_dict = arrival_bin[0]
ab_np = arrival_bin[1]
ab_dt = arrival_bin[2]

In [22]:
print(len(ab_dict))
for key in ab_dict.keys():
    print(key)

1
x_fixed


In [24]:
ab_dict['x_fixed']

Index(['start_days', 'holiday', 'dow0', 'dow1', 'dow2', 'dow3', 'dow4', 'dow5',
       'tf_slr_lstgs_open', 'tf_slr_lstgs',
       ...
       'focal_days', 'focal_holiday', 'focal_dow0', 'focal_dow1', 'focal_dow2',
       'focal_dow3', 'focal_dow4', 'focal_dow5', 'byr_us', 'byr_hist'],
      dtype='object', length=134)

In [26]:
for el in ab_dict['x_fixed']:
    print(el)

start_days
holiday
dow0
dow1
dow2
dow3
dow4
dow5
tf_slr_lstgs_open
tf_slr_lstgs
tf_slr_threads
tf_slr_slr_offers
tf_slr_byr_offers
tf_slr_accepts
tf_slr_accept_25
tf_slr_accept_50
tf_slr_accept_75
tf_slr_accept_100
tf_meta_lstgs_open
tf_meta_lstgs
tf_meta_threads
tf_meta_slr_offers
tf_meta_byr_offers
tf_meta_accepts
tf_meta_accept_25
tf_meta_accept_50
tf_meta_accept_75
tf_meta_accept_100
tf_leaf_lstgs_open
tf_leaf_lstgs
tf_leaf_threads
tf_leaf_slr_offers
tf_leaf_byr_offers
tf_leaf_accepts
tf_leaf_accept_25
tf_leaf_accept_50
tf_leaf_accept_75
tf_leaf_accept_100
tf_title_lstgs_open
tf_title_lstgs
tf_title_threads
tf_title_slr_offers
tf_title_byr_offers
tf_title_accepts
tf_title_accept_25
tf_title_accept_50
tf_title_accept_75
tf_title_accept_100
tf_cndtn_lstgs_open
tf_cndtn_lstgs
tf_cndtn_threads
tf_cndtn_slr_offers
tf_cndtn_byr_offers
tf_cndtn_accepts
tf_cndtn_accept_25
tf_cndtn_accept_50
tf_cndtn_accept_75
tf_cndtn_accept_100
relisted
store
slr_us
fast
photos
slr_bos
slr_lstgs
fdbk_scor

In [38]:
ab_args = {
    'id': 1,
    'outcome': 'days'
}
ab_params = utils.parse_params(ab_args)
print(ab_params)

hidden    1
Name: 1, dtype: int64


In [40]:
ab = FeedForward(len(ab_dict['x_fixed']), 1, 1)

In [64]:
ab.load_state_dict(torch.load('{}/{}_1.pt'.format(path, 'arrival_bin')))

RuntimeError: Error(s) in loading state_dict for FeedForward:
	Missing key(s) in state_dict: "seq.1.weight", "seq.1.bias". 
	Unexpected key(s) in state_dict: "seq.5.weight", "seq.5.bias". 
	size mismatch for seq.0.weight: copying a param with shape torch.Size([32, 134]) from checkpoint, the shape in current model is torch.Size([1, 134]).
	size mismatch for seq.0.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([1]).
	size mismatch for seq.3.weight: copying a param with shape torch.Size([32, 32]) from checkpoint, the shape in current model is torch.Size([1, 1]).
	size mismatch for seq.3.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([1]).

### Exploring RL Inputs

In [51]:
data_path = 'data/partitions/train_rl'
x_path = '{}/x.pkl'.format(data_path)
y_path = '{}/y.pkl'.format(data_path)
z_path = '{}/z.pkl'.format(data_path)

In [56]:
x = utils.unpickle(x_path)
y = utils.unpickle(y_path)
z = utils.unpickle(z_path)

In [58]:
print(type(x))
print(type(y))
print(type(z))

<class 'dict'>
<class 'dict'>
<class 'dict'>


In [61]:
for input_dict in [x, y, z]:
    print('New dict')
    for el in input_dict:
        print(el)

New dict
offer
thread
lstg
New dict
arrival
slr
byr
New dict
start
slr
byr


In [71]:
dict_types(x)

offer: <class 'pandas.core.frame.DataFrame'>
thread: <class 'pandas.core.frame.DataFrame'>
lstg: <class 'pandas.core.frame.DataFrame'>


In [72]:
dict_types(y)

arrival: <class 'dict'>
slr: <class 'dict'>
byr: <class 'dict'>


In [73]:
dict_types(z)

start: <class 'pandas.core.series.Series'>
slr: <class 'pandas.core.frame.DataFrame'>
byr: <class 'pandas.core.frame.DataFrame'>


In [75]:
for key, val in x.items():
    print(key)
    print(val.shape)
    print(val.columns)

offer
(653416, 35)
Index(['con', 'norm', 'reject', 'split', 'round', 'nines', 'msg', 'delay',
       'auto', 'exp', 'days', 'holiday', 'dow0', 'dow1', 'dow2', 'dow3',
       'dow4', 'dow5', 'minutes', 'slr_offers', 'slr_best', 'slr_offers_open',
       'slr_best_open', 'byr_offers', 'byr_best', 'byr_offers_open',
       'byr_best_open', 'slr_offers_diff', 'slr_best_diff',
       'slr_offers_open_diff', 'slr_best_open_diff', 'byr_offers_diff',
       'byr_best_diff', 'byr_offers_open_diff', 'byr_best_open_diff'],
      dtype='object')
thread
(178387, 2)
Index(['byr_us', 'byr_hist'], dtype='object')
lstg
(441457, 124)
Index(['start_days', 'holiday', 'dow0', 'dow1', 'dow2', 'dow3', 'dow4', 'dow5',
       'tf_slr_lstgs_open', 'tf_slr_lstgs',
       ...
       'meta29', 'meta30', 'meta31', 'meta32', 'meta33', 'meta34', 'new',
       'used', 'refurb', 'wear'],
      dtype='object', length=124)


In [76]:
dict_types(y['arrival'])

days: <class 'pandas.core.series.Series'>
loc: <class 'pandas.core.series.Series'>
hist: <class 'pandas.core.series.Series'>
bin: <class 'pandas.core.series.Series'>
sec: <class 'pandas.core.series.Series'>


In [77]:
dict_types(y['slr'])

delay: <class 'pandas.core.series.Series'>
accept: <class 'pandas.core.series.Series'>
reject: <class 'pandas.core.series.Series'>
con: <class 'pandas.core.series.Series'>
msg: <class 'pandas.core.series.Series'>
round: <class 'pandas.core.series.Series'>
nines: <class 'pandas.core.series.Series'>


In [78]:
dict_types(y['byr'])

delay: <class 'pandas.core.series.Series'>
accept: <class 'pandas.core.series.Series'>
reject: <class 'pandas.core.series.Series'>
con: <class 'pandas.core.series.Series'>
msg: <class 'pandas.core.series.Series'>
round: <class 'pandas.core.series.Series'>
nines: <class 'pandas.core.series.Series'>


In [81]:
z['start'].head()

lstg  thread  index
40    1       2        19646299
541   1       2        21236068
              3        21236070
              4        21236135
830   1       2        16672062
Name: clock, dtype: int64

In [82]:
for key in ['slr', 'byr']:
    val = z[key]
    print(key)
    print(val.shape)
    print(val.columns)

slr
(373556, 8)
Index(['slr_offers', 'slr_best', 'slr_offers_open', 'slr_best_open',
       'byr_offers', 'byr_best', 'byr_offers_open', 'byr_best_open'],
      dtype='object')
byr
(262207, 8)
Index(['slr_offers', 'slr_best', 'slr_offers_open', 'slr_best_open',
       'byr_offers', 'byr_best', 'byr_offers_open', 'byr_best_open'],
      dtype='object')
